# Int20h Test Assignement
by The Fractal Rhapsody

## Data Preproccesing

Importing the data:

In [ ]:
import numpy as np
import pandas as pd
import json

data = pd.read_csv('int20h-ds-test-dataset.csv')
data

,userid,user_state,event_name,event_attributes,event_created_date,event_platform,device_manufacture,device_model
0,c95c777785faec8dd910d019d7278ebe,CA,Add Vehicle Success,"{""Make"":""Dodge"",""Model"":""Caravan"",""Color"":""Whi...",2022-01-16 17:03:04,android,samsung,SM-N975U
1,c95c777785faec8dd910d019d7278ebe,CA,Add Vehicle Break,{},2022-01-16 17:07:47,android,samsung,SM-N975U
2,f344be2d9a042b7444f3cc5279e38ef1,FL,Calculator View,{},2022-01-16 17:16:25,android,samsung,SM-G973U1
3,c95c777785faec8dd910d019d7278ebe,CA,Add Payment Method Success,"{""Payment Method"":""Credit"",""Tokenized Pay"":""""}",2022-01-16 17:24:22,android,samsung,SM-N975U
4,e331ed81422d8fba55520a43a872e701,IL,Sign Up Success,"{""Method"":""Apple""}",2022-01-16 17:34:51,ios,Apple,"iPhone12,1"
...,...,...,...,...,...,...,...,...
23352,679eba26c4e75e0afb178360becfa21b,CA,Add Payment Method Success,"{""Payment Method"":""Credit"",""Tokenized Pay"":"""",...",2022-04-16 20:49:24,android,Google,Pixel 3a
23353,679eba26c4e75e0afb178360becfa21b,CA,Account Setup Profile Skip,"{""Screen"":""Address""}",2022-04-16 20:50:05,android,Google,Pixel 3a
23354,679eba26c4e75e0afb178360becfa21b,CA,Account Setup Profile Skip,"{""Screen"":""Phone Number""}",2022-04-16 20:50:10,android,Google,Pixel 3a
23355,679eba26c4e75e0afb178360becfa21b,CA,Chat Conversation Opened,"{""From"":""Dashboard"",""Transaction type"":""""}",2022-04-16 20:50:31,android,Google,Pixel 3a


Transforming event time to `pd.DateTime` format:

In [ ]:
data['event_created_date'] = pd.to_datetime(data.event_created_date)

Data overview:

In [ ]:
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23357 entries, 0 to 23356
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   userid              23357 non-null  object        
 1   user_state          23286 non-null  object        
 2   event_name          23357 non-null  object        
 3   event_attributes    23357 non-null  object        
 4   event_created_date  23357 non-null  datetime64[ns]
 5   event_platform      23357 non-null  object        
 6   device_manufacture  15847 non-null  object        
 7   device_model        15847 non-null  object        
dtypes: datetime64[ns](1), object(7)
memory usage: 1.4+ MB


<ipython-input-5-1c31c1101f98>:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  data.describe()


,userid,user_state,event_name,event_attributes,event_created_date,event_platform,device_manufacture,device_model
count,23357,23286,23357,23357,23357,23357,15847,15847
unique,3540,49,23,2309,20452,3,22,290
top,627f50253b42607513a1c93bb68201ad,CA,Order,"{""Premium Membership"":""True""}",2022-03-16 08:20:51,ios,Apple,"iPhone12,1"
freq,498,8627,6116,5451,17,9586,9586,1489
first,NaN,NaN,NaN,NaN,2022-01-15 02:06:43,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,2022-04-16 20:50:39,NaN,NaN,NaN


Parsing event attributes:

In [ ]:
data['event_attributes_d'] = data.event_attributes.apply(json.loads)

## Data Overview

Unique event categories:

In [ ]:
possible_events  = data.event_name.unique()
possible_events

array(['Add Vehicle Success', 'Add Vehicle Break', 'Calculator View',
       'Add Payment Method Success', 'Sign Up Success',
       'Add Payment Method Failed', 'Wallet Opened', 'Transaction Refund',
       'Email Confirmation Success', 'Account Setup Skip',
       'Subscription Premium', 'Chat Conversation Opened',
       'Account Setup Profile Skip',
       'Account History Transaction Details', 'Sign Out',
       'Chat Conversation Started', 'Order', 'Sign Up Error',
       'Subscription Premium Cancel', 'Calculator Used',
       'Subscription Premium Renew', 'Add Vehicle Failed',
       'Reset Password Set'], dtype=object)

In [ ]:
target_event = 'Subscription Premium Cancel'
# to Cancel sub we at first must have sub, presence of events
# Subscription Premium and Subscription Premium Renew means that condition is satisfied
# But it is possible that someone started long-term sub before our time range : 2022-01-15 02:06:43 - 2022-04-16 20:50:39	
# So lets look on event attributes:
for event in possible_events:
    print(f"event: {event} possible attributes combinations:\n")
    print("\n".join(data[data.event_name == event].event_attributes_d.apply(lambda x: ", ".join(sorted(list(set(x.keys()))))).unique()))
    print('----------')

event: Add Vehicle Success possible attributes combinations:

Color, End Date, Make, Model, Rental state, Start Date, Temporary, Year
Color, Make, Model, Temporary, Year
Color, End Date, Make, Model, Rental State, Start Date, Temporary, Year
----------
event: Add Vehicle Break possible attributes combinations:


----------
event: Calculator View possible attributes combinations:


----------
event: Add Payment Method Success possible attributes combinations:

Payment Method, Tokenized Pay
Payment Method
Payment Method, Promo code, Tokenized Pay
Payment Method, Promo code
----------
event: Sign Up Success possible attributes combinations:

Method
----------
event: Add Payment Method Failed possible attributes combinations:

Error Code, Error Message, Payment Method
Error Code, Error Message, Payment Method, Tokenized Pay
----------
event: Wallet Opened possible attributes combinations:


----------
event: Transaction Refund possible attributes combinations:

Amount, Id
----------
event:

From the cell above, we saw that the `Subscription Premium Cancel` event has no attributes. It could be beneficial to store some details for further analysis.

As we see, the `Order` event also contains some info about Premium. But since we have no information about what that application is, it is possible that the `Premium Membership` attribute belongs to another types of users.

In [ ]:
orders = data[data.event_name == "Order"]
orders.head()

,userid,user_state,event_name,event_attributes,event_created_date,event_platform,device_manufacture,device_model,event_attributes_d
72,0d51394c39dbf15021ccd09155ee6257,FL,Order,"{""Premium Membership"":""True""}",2022-01-15 09:23:48,outofband,NaN,NaN,{'Premium Membership': 'True'}
73,0d51394c39dbf15021ccd09155ee6257,FL,Order,"{""Premium Membership"":""True""}",2022-01-15 09:23:48,outofband,NaN,NaN,{'Premium Membership': 'True'}
74,0d51394c39dbf15021ccd09155ee6257,FL,Order,"{""Premium Membership"":""True""}",2022-01-15 10:16:12,outofband,NaN,NaN,{'Premium Membership': 'True'}
75,0d51394c39dbf15021ccd09155ee6257,FL,Order,"{""Premium Membership"":""True""}",2022-01-15 10:16:12,outofband,NaN,NaN,{'Premium Membership': 'True'}
94,4949c7b6dfa8cef23d641fd98d50ab18,TX,Order,"{""Premium Membership"":""True""}",2022-01-17 06:42:11,outofband,NaN,NaN,{'Premium Membership': 'True'}


In [ ]:
orders.event_attributes.unique()

array(['{"Premium Membership":"True"}', '{"Premium Membership":"False"}'],
      dtype=object)

The vast majority of orders belongs to Premium Membership:

In [ ]:
pd.pivot_table(orders[["event_attributes", "event_name"]], columns="event_attributes", aggfunc=len)

event_attributes,"{""Premium Membership"":""False""}","{""Premium Membership"":""True""}"
event_name,665,5451


In [ ]:
users_with_premium_order = data[data.event_attributes == '{"Premium Membership":"True"}'].userid.unique()
print(f"Number of users with Premium order: {len(users_with_premium_order)}")

Number of users with Premium order: 374


Now let's return to our Subscription Premium and Subscription Premium Renew events and look closely at the data:

In [ ]:
data[data.event_name == "Subscription Premium"].event_attributes_d.apply(lambda x: str(set(x.keys()))).unique()

array(["{'Start Date', 'Membership Type', 'Renewal Date'}"], dtype=object)

In [ ]:
data[data.event_name == "Subscription Premium Renew"].event_attributes_d.apply(lambda x: str(set(x.keys()))).unique()

array(["{'Start Date', 'Membership Type', 'Renewal Date'}"], dtype=object)

In [ ]:
pd.set_option("max_colwidth", None)
pd.set_option("max_rows", None)
data[data.event_name == 'Subscription Premium'].event_attributes_d.head(20)

34     {'Start Date': '2022-01-16', 'Renewal Date': '2022-02-17', 'Membership Type': 'Monthly'}
85     {'Start Date': '2022-01-16', 'Renewal Date': '2022-02-17', 'Membership Type': 'Monthly'}
119    {'Start Date': '2022-01-15', 'Renewal Date': '2022-02-16', 'Membership Type': 'Monthly'}
138    {'Start Date': '2022-01-15', 'Renewal Date': '2022-02-16', 'Membership Type': 'Monthly'}
153    {'Start Date': '2022-01-17', 'Renewal Date': '2022-02-18', 'Membership Type': 'Monthly'}
163    {'Start Date': '2022-01-17', 'Renewal Date': '2022-02-18', 'Membership Type': 'Monthly'}
172    {'Start Date': '2022-01-15', 'Renewal Date': '2022-02-16', 'Membership Type': 'Monthly'}
176    {'Start Date': '2022-01-15', 'Renewal Date': '2022-02-16', 'Membership Type': 'Monthly'}
323    {'Start Date': '2022-01-18', 'Renewal Date': '2022-02-19', 'Membership Type': 'Monthly'}
326    {'Start Date': '2022-01-18', 'Renewal Date': '2022-02-19', 'Membership Type': 'Monthly'}
336    {'Start Date': '2022-01-16', 'Ren

In [ ]:
data[data.event_name == 'Subscription Premium'].event_attributes.apply(lambda x: json.loads(x)["Membership Type"]).unique()

array(['Monthly'], dtype=object)

In [ ]:
data[data.event_name == 'Subscription Premium Renew'].event_attributes.apply(lambda x: json.loads(x)["Membership Type"]).unique()

array(['Yearly', 'Monthly'], dtype=object)

In [ ]:
# both events are semantically similar
# they may exist in not-overlapping time intervals (may be something on the app changed?)
data[data.event_name == 'Subscription Premium'].event_created_date.describe()

<ipython-input-19-775576dc7bc6>:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  data[data.event_name == 'Subscription Premium'].event_created_date.describe()


count                     711
unique                    704
top       2022-02-18 20:40:43
freq                        2
first     2022-01-15 13:58:45
last      2022-04-16 20:50:39
Name: event_created_date, dtype: object

In [ ]:
data[data.event_name == 'Subscription Premium Renew'].event_created_date.describe()

<ipython-input-20-1df01a8f1d3e>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  data[data.event_name == 'Subscription Premium Renew'].event_created_date.describe()


count                     310
unique                    225
top       2022-03-04 01:09:46
freq                        5
first     2022-01-31 01:14:26
last      2022-04-16 01:16:52
Name: event_created_date, dtype: object

As we can see, there are 4 sets of Premium users which exists simultanously:
<li>with Premium Subscribtion event
<li>with renewed monthly subscription
<li>with renewed yearly subscription
<li>with orders with ""Premium Membership":"True"" attribute

Now let's look on relationships beetween those sets

In [ ]:
users_with_renew_yearly = data.loc[data.event_name.isin([ 'Subscription Premium Renew']) & data.event_attributes.str.contains("Yearly"), "userid"].unique()
users_with_renew_monthly = data.loc[data.event_name.isin([ 'Subscription Premium Renew']) & data.event_attributes.str.contains("Monthly"), "userid"].unique()
users_with_renew = data.loc[data.event_name.isin([ 'Subscription Premium Renew']), "userid"].unique()
users_with_premimum = data.loc[data.event_name.isin(['Subscription Premium']), "userid"].unique()
users_with_premium_order = data[data.event_attributes == '{"Premium Membership":"True"}'].userid.unique()

print(f"Number of users with yearly renew: {len(users_with_renew_yearly)}")
print(f"Number of users with monthly renew: {len(users_with_renew_monthly)}")
print(f"Number of users with Premium Subscription event: {len(users_with_premimum)}")
print(f'Number of users with orders with ""Premium Membership":"True"" attribute: {len(users_with_renew_yearly)}')
print(f"Number of users: {len(data.userid.unique())}")

# len(users_with_renew_yearly), len(users_with_renew_monthly), len(users_with_premimum), len(users_with_premium_order), len(data.userid.unique())

Number of users with yearly renew: 110
Number of users with monthly renew: 123
Number of users with Premium Subscription event: 711
Number of users with orders with ""Premium Membership":"True"" attribute: 110
Number of users: 3540


In [ ]:
print(f"Number of users with both premium orders and subscription events: {len(np.intersect1d(users_with_premimum, users_with_premium_order))}")

Number of users with both premium orders and subscription events: 320


In [ ]:
print(f"Number of users with both yearly and monthly renew: {len(np.intersect1d(users_with_renew_yearly, users_with_renew_monthly))}")
print(f"Number of users with either yearly and monthly renew: {len(users_with_renew)}")
# renewed users cant be both simultanously

Number of users with both yearly and monthly renew: 0
Number of users with either yearly and monthly renew: 233


In [ ]:
print(f"Number of users with both yearly renew and subscription event: {len(np.intersect1d(users_with_renew_yearly, users_with_premimum))}")
print(f"Number of users with both monthly renew and subscription event: {len(np.intersect1d(users_with_renew_monthly, users_with_premimum))}")

Number of users with both yearly renew and subscription event: 70
Number of users with both monthly renew and subscription event: 98


In [ ]:
print(f"Number of users with both yearly renew and premium order event: {len(np.intersect1d(users_with_renew_yearly, users_with_premium_order))}")
print(f"Number of users with both monthly renew and premium order event: {len(np.intersect1d(users_with_renew_monthly, users_with_premium_order))}")

Number of users with both yearly renew and premium order event: 51
Number of users with both monthly renew and premium order event: 67


Both subsets of `users_with_renew` have approximately half of the users without any order, as well as mutual overlappings of subsets are a little bit messy, so below we will introduce following features:
<li> monthly subscription
<li> yearly subscription
<li> number of orders of user

and drop all the users who do not have premium orders or subscriptions

In [ ]:
users_with_any_premium = np.union1d(users_with_renew, users_with_premimum)
users_with_any_premium = np.union1d(users_with_any_premium, users_with_premium_order)
len(users_with_any_premium), len(data.userid.unique())

(799, 3540)

In [ ]:
# drop events of users without Premium
data_filtered = data[data.userid.isin(users_with_any_premium)]

# drop events, which are after the Cancel event
data_filtered_ = pd.DataFrame(columns=data_filtered.columns)
for user_id in users_with_any_premium:
    user_data = data_filtered[(data_filtered.userid == user_id)]
    dates = user_data[user_data.event_name == target_event ].event_created_date.values
    if len(dates) > 0:
        dates = max(dates)
        user_data = user_data[user_data.event_created_date <= dates]
    data_filtered_ = data_filtered_.append(user_data)

len(data_filtered_), len(data)

(14132, 23357)

In [ ]:
data_filtered = data_filtered_.copy()

Now lets look on transactional details of some possibly important transactions:

In [ ]:
pd.set_option("max_colwidth", None)
pd.set_option("max_rows", None)
data_filtered[data_filtered.event_attributes.str.contains('Error Message')][["event_name", "event_attributes"]]
#813	Add Payment Method Failed	{"Error Code":"ERROR-NaN10","Error Message":"Postal code can only contain letters,
#                                   numbers, spaces, and hyphens.","Payment Method":"Credit Card"}
# NAaN in Error Code is very very ODD, looks like there are bug in your database

,event_name,event_attributes
160,Add Payment Method Failed,"{""Error Code"":""ERROR-16010"",""Error Message"":""Card is debit but credit is required."",""Payment Method"":""Credit Card"",""Tokenized Pay"":""""}"
161,Add Payment Method Failed,"{""Error Code"":""ERROR-16010"",""Error Message"":""Card is debit but credit is required."",""Payment Method"":""Credit Card"",""Tokenized Pay"":""""}"
321,Add Payment Method Failed,"{""Error Code"":""ERROR-16010"",""Payment Method"":""Credit Card"",""Error Message"":""Card is debit but credit is required.""}"
701,Add Payment Method Failed,"{""Error Code"":""ERROR-16010"",""Error Message"":""Card is debit but credit is required."",""Payment Method"":""Credit Card"",""Tokenized Pay"":""""}"
813,Add Payment Method Failed,"{""Error Code"":""ERROR-NaN10"",""Error Message"":""Postal code can only contain letters, numbers, spaces, and hyphens."",""Payment Method"":""Credit Card""}"
1102,Add Payment Method Failed,"{""Error Code"":""ERROR-16010"",""Error Message"":""Card is debit but credit is required."",""Payment Method"":""Credit Card"",""Tokenized Pay"":""""}"
2010,Add Payment Method Failed,"{""Error Message"":""Card is debit but credit is required."",""Payment Method"":""Credit Card"",""Error Code"":""ERROR-16010""}"
2013,Add Payment Method Failed,"{""Error Code"":""ERROR-NaN10"",""Payment Method"":""Credit Card"",""Error Message"":""Payment method failed verification.""}"
2338,Add Payment Method Failed,"{""Error Code"":""ERROR-16010"",""Error Message"":""Card is debit but credit is required."",""Payment Method"":""Credit Card"",""Tokenized Pay"":""""}"
2339,Add Payment Method Failed,"{""Error Code"":""ERROR-16010"",""Error Message"":""Card is debit but credit is required."",""Payment Method"":""Credit Card"",""Tokenized Pay"":""""}"


As you can see below, all the refund amounts are negative. We will preserve the sign further:

In [ ]:
data_filtered[data_filtered.event_name.str.contains('Refund')][["userid","event_name", "event_attributes"]]

,userid,event_name,event_attributes
517,430d96d025d9c30dbd62aa4fd3591e98,Transaction Refund,"{""Id"":""8b58302d81c15a27545e492bdbeee2ef"",""Amount"":""-10""}"
2526,242f8255937541b03a117754ad36412d,Transaction Refund,"{""Id"":""570f2db617899393f6fe057112a4b894"",""Amount"":""-50""}"
3112,638a7b0cb6c8c36914929f0f6dec5f99,Transaction Refund,"{""Id"":""776d1e9a972a164f2758df79247dcf29"",""Amount"":""-20""}"
3276,829ecc5ca524bdfcfe1b1e7f21d1f9c7,Transaction Refund,"{""Id"":""350e9930fd6c550a087c5781bd1e661c"",""Amount"":""-10""}"
3278,83cf5f5241e85c4250279a9e47c140ba,Transaction Refund,"{""Id"":""cab720b1cb75ee0f2741cc09efb630be"",""Amount"":""-10""}"
3377,29c68f723b88e2d3bb1172f847b831e4,Transaction Refund,"{""Id"":""a06df3f8a4e3b4bb21ab0a01f186c3a7"",""Amount"":""-20""}"
3378,29c68f723b88e2d3bb1172f847b831e4,Transaction Refund,"{""Id"":""e12e7fca6bee6c15768229268fb37e5d"",""Amount"":""-0.96""}"
3379,29c68f723b88e2d3bb1172f847b831e4,Transaction Refund,"{""Id"":""0cc399c0a371c7a18779ed6eb7e42320"",""Amount"":""-20""}"
3826,258f464c03022a8510e9f4c067d1b8d7,Transaction Refund,"{""Id"":""abb700649f62a3d92ddcdf33e98862e1"",""Amount"":""-10""}"
3985,d13cf0b65d1701596269c4000bf647b8,Transaction Refund,"{""Id"":""a6f3727ea2d08ba23fceb7bc2a12c389"",""Amount"":""-20""}"


In [ ]:
data_filtered[data_filtered.event_name.str.contains('History')][["event_name", "event_attributes"]]

,event_name,event_attributes
55,Account History Transaction Details,"{""Type"":""Auto Payment"",""Navigated From"":""account_history""}"
139,Account History Transaction Details,"{""Type"":""Pro Plan Subscription"",""Navigated From"":""account_history"",""Status"":""Paid""}"
183,Account History Transaction Details,"{""Navigated From"":""Account History"",""Status"":""Pending"",""Type"":""Violation Transaction""}"
188,Account History Transaction Details,"{""Navigated From"":""Account History"",""Status"":""Pending"",""Type"":""Violation Transaction""}"
189,Account History Transaction Details,"{""Navigated From"":""Account History"",""Status"":""Pending"",""Type"":""Violation Transaction""}"
190,Account History Transaction Details,"{""Navigated From"":""Account History"",""Status"":""Paid"",""Type"":""Pro Plan Subscription""}"
284,Account History Transaction Details,"{""Navigated From"":""Account History"",""Status"":""Paid"",""Type"":""Order""}"
345,Account History Transaction Details,"{""Type"":""Auto Payment"",""Navigated From"":""account_history""}"
363,Account History Transaction Details,"{""Type"":""Pro Plan Subscription"",""Navigated From"":""account_history"",""Status"":""Paid""}"
364,Account History Transaction Details,"{""Type"":""Auto Payment"",""Navigated From"":""account_history""}"


In [ ]:
data_filtered[data_filtered.event_name.str.contains('Chat Conversation')][["userid","event_name", "event_attributes"]].head(50)
# looks like Chat Conversation Started always follows Opened

,userid,event_name,event_attributes
37,017f4557810b3ba165d2959c17b916b6,Chat Conversation Opened,"{""From"":""Menu""}"
40,017f4557810b3ba165d2959c17b916b6,Chat Conversation Opened,"{""From"":""Menu""}"
41,017f4557810b3ba165d2959c17b916b6,Chat Conversation Opened,"{""From"":""Menu""}"
42,017f4557810b3ba165d2959c17b916b6,Chat Conversation Opened,"{""From"":""Menu""}"
43,017f4557810b3ba165d2959c17b916b6,Chat Conversation Opened,"{""From"":""Menu""}"
44,017f4557810b3ba165d2959c17b916b6,Chat Conversation Opened,"{""From"":""Menu""}"
45,017f4557810b3ba165d2959c17b916b6,Chat Conversation Opened,"{""From"":""Menu""}"
46,017f4557810b3ba165d2959c17b916b6,Chat Conversation Opened,"{""From"":""Menu""}"
76,017f4557810b3ba165d2959c17b916b6,Chat Conversation Started,{}
109,ddfd6a4298e15df738efd4daa76403e4,Chat Conversation Opened,"{""From"":""Menu"",""Transaction type"":""""}"


In [ ]:
# event history of some user
data_filtered[data_filtered.userid == "520c53e60a665c9c1c78671024f640a7"]

,userid,user_state,event_name,event_attributes,event_created_date,event_platform,device_manufacture,device_model,event_attributes_d
1014,520c53e60a665c9c1c78671024f640a7,TX,Sign Up Success,"{""Method"":""Google""}",2022-01-20 19:27:15,android,samsung,SM-N985F,{'Method': 'Google'}
1015,520c53e60a665c9c1c78671024f640a7,TX,Add Payment Method Success,"{""Payment Method"":""Credit"",""Tokenized Pay"":""Google Pay""}",2022-01-20 19:36:42,android,samsung,SM-N985F,"{'Payment Method': 'Credit', 'Tokenized Pay': 'Google Pay'}"
1016,520c53e60a665c9c1c78671024f640a7,TX,Subscription Premium,"{""Start Date"":""2022-01-20"",""Renewal Date"":""2022-02-21"",""Membership Type"":""Monthly""}",2022-01-20 19:37:26,outofband,NaN,NaN,"{'Start Date': '2022-01-20', 'Renewal Date': '2022-02-21', 'Membership Type': 'Monthly'}"
1058,520c53e60a665c9c1c78671024f640a7,TX,Account History Transaction Details,"{""Navigated From"":""Account History"",""Status"":""Pending"",""Type"":""Auto Payment""}",2022-01-21 01:48:19,android,samsung,SM-N985F,"{'Navigated From': 'Account History', 'Status': 'Pending', 'Type': 'Auto Payment'}"
1059,520c53e60a665c9c1c78671024f640a7,TX,Wallet Opened,{},2022-01-21 01:49:44,android,samsung,SM-N985F,{}
1060,520c53e60a665c9c1c78671024f640a7,TX,Calculator View,{},2022-01-21 01:52:14,android,samsung,SM-N985F,{}
1061,520c53e60a665c9c1c78671024f640a7,TX,Chat Conversation Opened,"{""From"":""Menu"",""Transaction type"":""""}",2022-01-21 01:53:06,android,samsung,SM-N985F,"{'From': 'Menu', 'Transaction type': ''}"
1062,520c53e60a665c9c1c78671024f640a7,TX,Chat Conversation Started,{},2022-01-21 01:53:06,android,samsung,SM-N985F,{}
1063,520c53e60a665c9c1c78671024f640a7,TX,Chat Conversation Started,{},2022-01-21 01:54:21,android,samsung,SM-N985F,{}
1064,520c53e60a665c9c1c78671024f640a7,TX,Chat Conversation Started,{},2022-01-21 01:57:21,android,samsung,SM-N985F,{}


Let's look on event attributes for several events types

In [ ]:
data_filtered[data_filtered.event_name.str.contains('Chat Conversation Started')][["event_name", "event_attributes"]].event_attributes.unique()

array(['{}'], dtype=object)

In [ ]:
data_filtered[data_filtered.event_name.str.contains('Chat Conversation Opened')][["event_name", "event_attributes"]].event_attributes.unique()

array(['{"From":"Menu"}', '{"From":"Menu","Transaction type":""}',
       '{"Transaction type":"AutoPayment","From":"Transaction Details"}',
       '{"From":"Transaction Details","Transaction type":"AutoPayment"}',
       '{"From":"Transaction Details","Transaction type":"Pro Plan Subscription"}',
       '{}',
       '{"Transaction type":"ManualPayment","From":"Transaction Details"}',
       '{"From":"Transaction Details","Transaction type":"OrderPayment"}',
       '{"Transaction type":"OrderAlert","From":"Transaction Details"}',
       '{"Transaction type":"Pro Plan Subscription","From":"Transaction Details"}',
       '{"From":"Deeplink"}',
       '{"From":"Transaction Details","Transaction type":"OrderAlert"}',
       '{"From":"Transaction Details","Transaction Type":"Pro Plan Subscription"}',
       '{"From":"Deeplink","Transaction type":""}',
       '{"From":"","Transaction type":""}',
       '{"Transaction type":"OrderPayment","From":"Transaction Details"}',
       '{"Transaction 

In [ ]:
len(data_filtered[data_filtered.event_name.str.contains('Chat Conversation Opened')]), len(data_filtered[data_filtered.event_name.str.contains('Chat Conversation Started')])

(1121, 894)

In [ ]:
data_filtered[data_filtered.event_name.str.contains('Add Payment Method Success')][["event_name", "event_attributes"]].event_attributes.unique()

array(['{"Tokenized Pay":"Apple Pay","Payment Method":"Debit"}',
       '{"Payment Method":"Debit"}',
       '{"Payment Method":"PayPal","Tokenized Pay":""}',
       '{"Payment Method":"Credit","Tokenized Pay":"Google Pay"}',
       '{"Payment Method":"Debit","Tokenized Pay":""}',
       '{"Tokenized Pay":"Apple Pay","Payment Method":"Credit"}',
       '{"Payment Method":"Venmo"}', '{"Payment Method":"PayPal"}',
       '{"Payment Method":"Debit","Tokenized Pay":"Google Pay"}',
       '{"Payment Method":"Credit"}',
       '{"Payment Method":"Credit","Tokenized Pay":""}',
       '{"Payment Method":"Debit","Tokenized Pay":"Google Pay","Promo code":""}',
       '{"Payment Method":"Debit","Tokenized Pay":"","Promo code":""}',
       '{"Payment Method":"Debit","Promo code":""}',
       '{"Payment Method":"Credit","Promo code":""}',
       '{"Payment Method":"PayPal","Tokenized Pay":"","Promo code":""}',
       '{"Tokenized Pay":"Apple Pay","Payment Method":"Debit","Promo code":""}',
       '

## Feature extraction

Build pivot tables of occurences/counts of each event for each user for correlation analysis:

In [ ]:
user_event_featuremap_occurence = pd.pivot_table(data_filtered[['userid', 'event_name']], index='userid', columns='event_name', aggfunc=lambda x: 1 if len(x) > 0 else 0).fillna(0)
user_event_featuremap_occurence.head()

event_name,Account History Transaction Details,Account Setup Profile Skip,Account Setup Skip,Add Payment Method Failed,Add Payment Method Success,Add Vehicle Break,Add Vehicle Failed,Add Vehicle Success,Calculator Used,Calculator View,...,Order,Reset Password Set,Sign Out,Sign Up Error,Sign Up Success,Subscription Premium,Subscription Premium Cancel,Subscription Premium Renew,Transaction Refund,Wallet Opened
userid,,,,,,,,,,,,,,,,,,,,,
0006869712ec9841dc36234bce245203,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
00596aa4edee32815aaf4aad659c484e,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
007b54670d72df309afd87befcbb63db,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
0136a8eeca5d16ac1a521dd0e27ecec0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
01529f53aaba5b512168d432d5650556,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
user_event_featuremap_counts = pd.pivot_table(data_filtered[['userid', 'event_name']], index='userid', columns='event_name', aggfunc=len).fillna(0)
user_event_featuremap_counts.head()

event_name,Account History Transaction Details,Account Setup Profile Skip,Account Setup Skip,Add Payment Method Failed,Add Payment Method Success,Add Vehicle Break,Add Vehicle Failed,Add Vehicle Success,Calculator Used,Calculator View,...,Order,Reset Password Set,Sign Out,Sign Up Error,Sign Up Success,Subscription Premium,Subscription Premium Cancel,Subscription Premium Renew,Transaction Refund,Wallet Opened
userid,,,,,,,,,,,,,,,,,,,,,
0006869712ec9841dc36234bce245203,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
00596aa4edee32815aaf4aad659c484e,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,...,3.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
007b54670d72df309afd87befcbb63db,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
0136a8eeca5d16ac1a521dd0e27ecec0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0
01529f53aaba5b512168d432d5650556,2.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


We analysed the attributes of all of the event types and chose the features, which can correlate with the target event. Here we are extracting such features:

In [ ]:
# attr from events
# events
has_cancel_by_userid = pd.pivot_table(data_filtered[['userid', 'event_name']], index='userid', columns='event_name', aggfunc=lambda x: 1 if len(x) > 0 else 0).fillna(0)[[target_event]]

amount_data = pd.DataFrame(data=data_filtered[['userid']], columns=['userid'])
amount_data['Refund amount'] = data_filtered.event_attributes_d.apply(lambda x: float(x['Amount']) if 'Amount' in x.keys() else None)
amount_data = amount_data.groupby(by='userid').sum()

subscription_type_data = pd.DataFrame(data=data_filtered[['userid']], columns=['userid'])
subscription_type_data['Subscription type'] = data_filtered.event_attributes_d.apply(lambda x: x['Membership Type'] if 'Membership Type' in x.keys() else None)
subscription_type_data_yearly = subscription_type_data.groupby(by='userid').agg(lambda x: 1 if 'Yearly' in x.values else 0)
subscription_type_data_monthly = subscription_type_data.groupby(by='userid').agg(lambda x: 1 if 'Monthly' in x.values else 0)
subscription_type_data = subscription_type_data_yearly.merge(subscription_type_data_monthly, left_index=True, right_index=True)
subscription_type_data = subscription_type_data.rename(columns={'Subscription type_x': 'Yearly subscription', 'Subscription type_y': 'Monthly subscription'})

promocode_data = pd.DataFrame(data=data_filtered[['userid']], columns=['userid'])
promocode_data['Has promo code'] = data_filtered.event_attributes_d.apply(lambda x: 1 if 'Promo code' in x.keys() and len(x['Promo code']) else 0)
promocode_data = promocode_data.groupby(by='userid').max()

chat_opened_from_data = pd.DataFrame(data=data_filtered[['userid']], columns=['userid'])
chat_opened_from_data['Chat Conversation Opened From'] = data_filtered.event_attributes_d.apply(
    lambda x: x['From'] if 'From' in x.keys() and len(x['From']) else None
)
chat_opened_from_data = chat_opened_from_data.set_index('userid')
chat_opened_from_data = pd.get_dummies(chat_opened_from_data, prefix_sep=' ')
chat_opened_from_data = chat_opened_from_data.groupby(by='userid').sum()

# merges
attributes_data_by_user = amount_data.copy()
attributes_data_by_user = attributes_data_by_user.merge(subscription_type_data, left_index=True, right_index=True)
attributes_data_by_user = attributes_data_by_user.merge(promocode_data, left_index=True, right_index=True)
attributes_data_by_user = attributes_data_by_user.merge(chat_opened_from_data, left_index=True, right_index=True)
attributes_data_by_user = attributes_data_by_user.merge(has_cancel_by_userid, left_index=True, right_index=True)

In [ ]:
attributes_data_by_user.head()

,Refund amount,Yearly subscription,Monthly subscription,Has promo code,Chat Conversation Opened From Dashboard,Chat Conversation Opened From Deeplink,Chat Conversation Opened From Menu,Chat Conversation Opened From Transaction Details,Subscription Premium Cancel
userid,,,,,,,,,
0006869712ec9841dc36234bce245203,0.0,1,1,0,0,0,0,0,0.0
00596aa4edee32815aaf4aad659c484e,0.0,0,1,0,0,0,0,0,0.0
007b54670d72df309afd87befcbb63db,0.0,0,1,0,2,0,2,0,0.0
0136a8eeca5d16ac1a521dd0e27ecec0,0.0,0,1,0,0,0,0,0,0.0
01529f53aaba5b512168d432d5650556,0.0,0,1,0,0,0,0,0,0.0


## Correlations computation

### General 

In [ ]:
user_event_featuremap_occurence.corr()[[target_event]].sort_values(by=target_event)

event_name,Subscription Premium Cancel
event_name,
Subscription Premium Renew,-0.207159
Order,-0.171091
Add Payment Method Failed,-0.028738
Reset Password Set,-0.023057
Sign Out,-0.017172
Add Vehicle Failed,-0.007425
Calculator Used,-0.002639
Email Confirmation Success,0.009060
Sign Up Error,0.022141


In [ ]:
user_event_featuremap_counts.corr(method='spearman')[[target_event]].sort_values(by=target_event)

event_name,Subscription Premium Cancel
event_name,
Subscription Premium Renew,-0.206757
Order,-0.186232
Add Payment Method Failed,-0.031412
Reset Password Set,-0.022951
Sign Out,-0.017377
Add Vehicle Failed,-0.008356
Add Vehicle Success,-0.003593
Calculator Used,0.000297
Email Confirmation Success,0.012027


The user intending to write to Help Desk will likely cancel his subscription. <br>
Interest to Balance Sheet [ e.g., Wallet Opened / Transactional Refund ] <br>
also negative sign [may be an expression of user`s unsatisfaction of service] <br>

Active [high number of orders] and loyal [subscription renew] users are naturally less prone to Subscription Premium Cancel

In [ ]:
attributes_data_by_user.corr(method='spearman')[[target_event]].sort_values(by=target_event)

,Subscription Premium Cancel
Refund amount,-0.267861
Yearly subscription,-0.085511
Has promo code,-0.017656
Chat Conversation Opened From Dashboard,-0.016683
Chat Conversation Opened From Deeplink,0.052512
Monthly subscription,0.058552
Chat Conversation Opened From Transaction Details,0.129043
Chat Conversation Opened From Menu,0.473684
Subscription Premium Cancel,1.000000


We can see that Subscription Premium Cancel higly relatated with Refunds(**reminder: amount is negative number!**) <br>
A yearly subscription is a tiny bit better than a monthly (maybe because a long-term subscription is a sign of firm intention to use the app)<br>
Opening Chat Conversation from Menu is a signal for anxiety about user loyalty <br>

### Analysis by regions

In [ ]:
data_filtered[['userid', 'user_state']].groupby(by='user_state').nunique().sort_values(by='userid', ascending=False)

,userid
user_state,
CA,342
TX,192
FL,63
IL,41
NY,34
VA,26
NJ,17
MD,10
MA,8


We can see, that solid amount of data comes from CA and TX states only.
Let's compare them with the whole dataset.

In [ ]:
def event_occurence_featuremap_correlation(df, target):
    user_event_featuremap = pd.pivot_table(df[['userid', 'event_name']], index='userid', columns='event_name', aggfunc=lambda x: 1 if len(x) > 0 else 0).fillna(0)
    return user_event_featuremap.corr()[[target]].sort_values(by=target)

In [ ]:
def event_counts_featuremap_correlation(df, target, method='spearman'):
    user_event_featuremap_counts = pd.pivot_table(df[['userid', 'event_name']], index='userid', columns='event_name', aggfunc=len).fillna(0)
    return user_event_featuremap_counts.corr(method=method)[[target]].sort_values(by=target)

#### CA

In [ ]:
ca_data = data_filtered[data_filtered.user_state == 'CA'].copy()
event_occurence_featuremap_correlation(ca_data, target_event)

event_name,Subscription Premium Cancel
event_name,
Subscription Premium Renew,-0.212814
Order,-0.169417
Add Vehicle Failed,-0.036790
Sign Out,-0.024209
Add Payment Method Failed,-0.012278
Email Confirmation Success,0.004060
Calculator Used,0.010369
Add Vehicle Break,0.014122
Sign Up Error,0.079711


In [ ]:
event_counts_featuremap_correlation(ca_data, target_event)

event_name,Subscription Premium Cancel
event_name,
Subscription Premium Renew,-0.198920
Order,-0.180736
Add Vehicle Failed,-0.036491
Sign Out,-0.020070
Add Payment Method Failed,-0.015614
Email Confirmation Success,0.009669
Calculator Used,0.015507
Add Vehicle Break,0.025230
Add Vehicle Success,0.045176


#### TX

In [ ]:
tx_data = data_filtered[data_filtered.user_state == 'TX'].copy()
event_occurence_featuremap_correlation(tx_data, target_event)

event_name,Subscription Premium Cancel
event_name,
Order,-0.182404
Subscription Premium Renew,-0.162557
Email Confirmation Success,-0.157985
Add Payment Method Failed,-0.069697
Sign Out,-0.059358
Calculator View,-0.008842
Add Vehicle Success,-0.001945
Subscription Premium,0.001685
Account Setup Skip,0.009112


In [ ]:
event_counts_featuremap_correlation(tx_data, target_event)

event_name,Subscription Premium Cancel
event_name,
Subscription Premium Renew,-0.179407
Order,-0.179375
Email Confirmation Success,-0.148580
Add Payment Method Failed,-0.070505
Sign Out,-0.063831
Add Vehicle Success,-0.046124
Calculator View,-0.007424
Subscription Premium,0.002806
Account Setup Skip,0.008650


### Analysis by platform

In [ ]:
data_filtered.event_platform.unique()

array(['android', 'outofband', 'ios'], dtype=object)

In [ ]:
user_to_os = pd.pivot_table(
    data_filtered[['userid', 'event_platform']], 
    index='userid', 
    columns='event_platform', 
    aggfunc=lambda x: 1 if len(x) > 0 else 0
).fillna(0).drop(columns='outofband')

In [ ]:
pd.merge(
    pd.pivot_table(
        data_filtered[['userid', 'event_name']], 
        index='userid', 
        columns='event_name', 
        aggfunc=lambda x: 1 if len(x) > 0 else 0
    ).fillna(0), 
    user_to_os, 
    left_index=True, 
    right_index=True
).corr()[['android', 'ios']]

,android,ios
Account History Transaction Details,0.039301,0.038362
Account Setup Profile Skip,0.012029,0.010498
Account Setup Skip,-0.024634,0.034704
Add Payment Method Failed,0.041943,-0.022148
Add Payment Method Success,0.089164,0.136874
Add Vehicle Break,-0.102296,0.126336
Add Vehicle Failed,-0.052675,0.056626
Add Vehicle Success,0.058670,0.126055
Calculator Used,-0.018644,0.046056
Calculator View,-0.022231,0.069789


In [ ]:
pd.merge(
    pd.pivot_table(
        data_filtered[['userid', 'event_name']], 
        index='userid', 
        columns='event_name', 
        aggfunc=len
    ).fillna(0), 
    user_to_os, 
    left_index=True, 
    right_index=True
).corr(method='spearman')[['android', 'ios']]

,android,ios
Account History Transaction Details,0.074704,0.002833
Account Setup Profile Skip,0.012575,0.009935
Account Setup Skip,-0.024744,0.034812
Add Payment Method Failed,0.040046,-0.020298
Add Payment Method Success,0.070557,0.119675
Add Vehicle Break,-0.102801,0.126784
Add Vehicle Failed,-0.052675,0.056626
Add Vehicle Success,0.070778,0.084086
Calculator Used,-0.018831,0.046784
Calculator View,-0.005454,0.054265


As we can see, the dissimilarity between users on different operating systems is insignificant. Only `Sign Up Success` and `Sign Out` are slightly different, probably due to the typical authentification mechanism (e.g., AppleID vs. Google Account).

## Conclusion

The user intending to write to Help Desk will likely cancel his subscription. <br>
Interest to Balance Sheet [ e.g., Wallet Opened / Transactional Refund ] <br>
also negative sign [may be an expression of user`s unsatisfaction of service] <br>

Active [high number of orders] and loyal [subscription renew] users are naturally less prone to Subscription Premium Cancel

We can see that Subscription Premium Cancel higly relatated with Refunds(**reminder: amount is negative number!**) <br>
A yearly subscription is a tiny bit better than a monthly (maybe because a long-term subscription is a sign of firm intention to use the app)<br>
Opening Chat Conversation from Menu is a signal for anxiety about user loyalty <br>

Due to our analysis, regions of users and platforms are insignificant <br>

